In [ ]:
pip install langchain openai gradio fastapi uvicorn


In [ ]:
!pip install --upgrade langchain langchain-community


In [ ]:
!pip install -U langchain-ollama

In [1]:
import uvicorn
import requests
import ollama
import gradio as gr
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from langchain_ollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage

# Initialize FastAPI
app = FastAPI()

# LangChain Chat Model
chat_model = ChatOllama(model="mistral")

def get_latest_phones():
    """Fetches latest phone models from the API and filters them using AI."""
    PHONE_API_URL = "http://127.0.0.1:8000/latest"
    
    try:
        response = requests.get(PHONE_API_URL, timeout=5)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        raise HTTPException(status_code=500, detail=f"Error fetching phone data: {str(e)}")
    
    phone_models = data.get("latest_models", [])
    if not phone_models:
        return {"error": "No phone models available"}
    
    phone_text = "\n".join([
        f"Brand: {p.get('brand', 'Unknown')}, Model: {p.get('model', 'Unknown')}, Release Date: {p.get('release_date', 'Unknown')}" 
        for p in phone_models
    ])
    
    messages = [
        SystemMessage(content="You are an AI that selects the most recent phone model."),
        HumanMessage(content=f"Given the following phone models, identify the latest one:\n{phone_text}")
    ]
    
    response = chat_model(messages)
    
    return {"latest_phone_filtered_by_ai": response.content}

@app.get("/latest")
def fetch_latest_phone():
    return get_latest_phones()

def chatbot(query):
    """Processes user query using LangChain's ChatOllama."""
    messages = [
        SystemMessage(content="You are a phone assistant."),
        HumanMessage(content=query)
    ]
    
    response = chat_model(messages)
    return response.content

# Gradio UI
with gr.Blocks() as ui:
    gr.Markdown("##  AI-Powered Phone Chatbot")
    with gr.Row():
        query_input = gr.Textbox(label="Ask about phones:")
        response_output = gr.Textbox(label="Chatbot Response", interactive=False)
    
    ask_button = gr.Button("Ask AI")
    ask_button.click(chatbot, inputs=[query_input], outputs=[response_output])
    
    fetch_button = gr.Button(" Get Latest Phones")
    phone_output = gr.Textbox(label="Latest Phone Model", interactive=False)
    
    fetch_button.click(fn=lambda: get_latest_phones()["latest_phone_filtered_by_ai"], inputs=[], outputs=[phone_output])

# Run FastAPI & Gradio UI
if __name__ == "__main__":
    import threading
    threading.Thread(target=lambda: uvicorn.run(app, host="127.0.0.1", port=8003, log_level="info"), daemon=True).start()
    ui.launch(server_name="127.0.0.1", server_port=7860)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\HP\AppData\Local\Temp\ipykernel_8556\2127014202.py:56: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat_model(messages)
